# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.
Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откроем и изучим файл

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Общее количество записей 3214.

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую.

Размеры тестового и валидационного наборов обычно равны. Исходные данные разбиваем в соотношении 3:1:1.

Целевой признак - столбец `is_ultra`. Признак - категориальный, т.е. перед нами задача классификации.
- 0 - клиенту больше подходит тариф `smart`, 
- 1 - клиенту больше подходит тариф `ultra`.

## 2. Разделим данные на выборки

In [5]:
# создадим наборы данных features (признаки) и target (целевой признак).
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

In [6]:
# разделим данные features и target на две части (train и test) в соотношении 60% к 40% 
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [7]:
# разделим данные features_test и target_test на две части (test и valid) в соотношении 50% к 50% 
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size=0.5, random_state=12345)

In [8]:
for i in [features_train, features_test, features_valid]:
    print(i.shape)

(1928, 4)
(643, 4)
(643, 4)


Данные разбились правильно.

## 3. Исследуем модели

Рассмотрим три модели:
- `DecisionTreeClassifier` (Решающее дерево)
- `RandomForestClassifier` (Случайный лес)
- `LogisticRegression` (Логистическая регрессия)

### 3.1 `DecisionTreeClassifier` (Решающее дерево)

In [9]:
best_result = 0
for depth in range(1, 50): 
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    if best_result <= accuracy:
        best_result = accuracy
        best_depth = depth
print('Лучший результат =', best_result, 'при depth = ', best_depth)

Лучший результат = 0.7993779160186625 при depth =  7


### 3.2 `RandomForestClassifier` (Случайный лес)

In [10]:
best_result = 0
for depth in range(9,10): 
    for estimators in range(1,52,5):
        model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if best_result <= accuracy:
            best_result = accuracy
            best_depth = depth
            best_n_estimators = estimators
print('Лучший результат =', best_result, 'при depth = ', best_depth, 'и при n_estimators = ', estimators)

Лучший результат = 0.8133748055987559 при depth =  9 и при n_estimators =  51


### 3.3 `LogisticRegression` (Логистическая регрессия)

In [11]:
best_result = 0
for solver in ['newton-cg','lbfgs','sag','saga','liblinear']:
    for penalty in ['l1', 'l2', 'elasticnet', 'none']
        model = LogisticRegression(random_state=12345, solver = solver, max_iter=4000)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if best_result <= accuracy:
            best_result = accuracy
            best_solver = solver
            best_penalty = penalty

print('Лучший результат =', best_result, 'при solver = ',best_solver, 'и penalty =', best_penalty)

Лучший результат = 0.7402799377916018 при solver =  liblinear


#### Наилучший результат показала модель `RandomForestClassifier` (Случайный лес). Проверим ее на тестовой выборке.

## 4. Проверим модель `RandomForestClassifier` (Случайный лес) на тестовой выборке

In [12]:
model = RandomForestClassifier(n_estimators=51, max_depth=9, random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions_valid)
print(accuracy)

0.7947122861586314


Модель `RandomForestClassifier` (Случайный лес) на тестовой выборке показала точность соизмеримую с точностью на валидационной выборке. Модель предположительно не имеет проблемы переобучения. Значение `accuracy > 0.75`.

## 5. (бонус) Проверим модель на адекватность

In [13]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [14]:
2229/3214

0.693528313627878

Если сравнить значения `is_ultra` с константой равной 0, то доля правильных ответов составит <b>69,35%</b>. Наша модель `RandomForestClassifier` (Случайный лес) дает на 10% результат точнее, поэтому ее можно рекомендовать к применению на практике.